This file instantiates objects of the class Particle in the particle.ipynb file, using initial velocities and initial positions generated in the sampling.ipynb file.

Author: 18BME2104 Kaushal

In [1]:
from inspect import getmembers, isfunction, isclass # To inspect imported modules
from IPython.display import display, Math, Latex, clear_output # To use latex typesetting style

import numpy as np # For computations
import sympy as sm # For symbolic use

import pandas as pd # To use large tables
import matplotlib.pyplot as plt #To make plots
import csv # To read csv file of position and velocities required to initialize particles

In [ ]:
# Probably good idea to make plots, write to csv and other things in another notebook. Let's see.

In [3]:
class Interaction:
    def __init__(self, particles, fields):
        self.particles = particles
        self.fields = fields
        
        # particle.update(field)
        
    def __string__(self):
        pass
        
    def docstring(self):
            pass
            # print (something)
        
    def lorentz(self):
        E, B, v = sm.symbols('E B v')
        force = E + sm.physics.cross(v,B)
        return force
    

In [ ]:
#create many particles
# by sampling different positions and velocities using various distribution functions
#Loop
    #get fields at their position
    #move the particles
    
#create new set of particles after some time, to model flow of particles/ionized gas 
#in a magnetron sputtering chamber

#For old particles that move out of the trap bounds (position exceeding some limits), or 
#modelingf particles being absorbed in the sputtered coating
#with an associated probability of absorption
#just delete those particles from the collection
#(but can be saved in csv files, so that they can stay recorded to be reused later)